# Στειροπούλου Ευαγγελία 
### Α.Μ. 111520180016
## Εργασία 2η Αναγνώριση Προτύπων - Μηχανική Μάθηση



# Face recognition: 

In [1]:
!pip3 install numpy
!pip3 install matplotlib
!pip3 install -U scikit-learn

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import glob
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, ConfusionMatrixDisplay, classification_report
# from google.colab import drive
# drive.mount('/content/gdrive')

  Using cached numpy-1.24.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached matplotlib-3.7.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
  Using cached contourpy-1.0.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
     |████████████████████████████████| 3.3 MB 2.5 MB/s eta 0:00:01
  Using cached kiwisolver-1.4.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached fonttools-4.39.4-py3-none-any.whl (1.0 MB)
Processing /home/eva/.cache/pip/wheels/44/08/18/d0b86f591e929e063b3134b126c8a77b3758e527fe1a3f6fb8/sklearn-0.0.post5-py3-none-any.whl


ModuleNotFoundError: No module named 'sklearn'

## I: loadImages function

In [ ]:
def loadImages(path, set_number):
    image_paths = glob.glob(path + "/person*_*.png")

    image_matrix = []
    labels = []
    for path in image_paths:
        filename = path.split('/')[-1]
        person_id = int(filename.split('_')[0][6:])
        image_number = int(filename.split('_')[1].split('.')[0])

        if ((set_number == "Set_1" and 1 <= image_number <= 7) or
            (set_number == "Set_2" and 8 <= image_number <= 19) or
            (set_number == "Set_3" and 20 <= image_number <= 31) or
            (set_number == "Set_4" and 32 <= image_number <= 45) or
            (set_number == "Set_5" and 46 <= image_number <= 64)):

            image_array = np.array(plt.imread(path).flatten())
            image_matrix.append(image_array)
            labels.append(person_id)
    
    return np.array(image_matrix), labels


Όπως αναφέρεται στην εκφώνηση, η προεπεξεργασία της εικόνας είναι απαραραίτητη, και γίνεται με την αφαίρεση της μέσης τιμής της και στην συνέχεια με την διαίρεση με την τυπική απόκλιση των τιμών της.

In [ ]:
def normalization(faces):
    X = np.ndarray(shape=faces.shape)  # Normalized data matrix
    for i in range(faces.shape[0]):
        X[i] = (faces[i] - np.mean(faces, axis=0)) / np.std(faces, axis=0)
    return X

## II: Train Eigenfaces method

In [ ]:
X_set1, y_set1 = loadImages("/home/eva/Documents/machine_learning/project2/faces", "Set_1")
   
X_train_set_1 = normalization(X_set1) #Train the model with the first set of images

#for dimensions 9 and 30
dimensions = [9, 30]
#create a dictionary to store the results of the accuracy for every set of images
accuracy = {}
for d in dimensions:
    print("Dimension: " + str(d))
    pca = PCA(n_components=d, whiten=True) #Apply PCA
    X_train_pca = pca.fit_transform(X_train_set_1) #Fit the model using the initial training set(set 1)
    knn = KNeighborsClassifier(n_neighbors=1).fit(X_train_pca, y_set1)  #Train the model
    for set_number in range(1,6):
        #Test the model, for every set of images
        X_set, y_set = loadImages("/home/eva/Documents/machine_learning/project2/faces", "Set_" + str(set_number))
        X_test_set = normalization(X_set)
        X_test_set = pca.transform(X_test_set)
        y_pred = knn.predict(X_test_set)
        accuracy[str(set_number)] = accuracy_score(y_set, y_pred)*100
        #print("Accuracy for dimension " + str(d) + " is: " + str(accuracy))

        #Plot the confusion matrix for the set of images
        cm = confusion_matrix(y_set, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_set))
        disp.plot()
        print("Confusion matrix")
        plt.title("Accuracy for set " + str(set_number) + " is " + str(accuracy[str(set_number)]) + "%")
        plt.show()

Παρατηρώ ότι για d = 30, η ακρίβεια ταξινόμησης είναι υψηλότερη για τα υπόλοιπα sets****************************. 

## III: 9 top eigenvectors

In [ ]:
#plot the top 9 eigenfaces
plt.figure(figsize=(9, 8))
for i in range(9):  
    plt.subplot(3, 3, i + 1)
    eigenfaces = pca.components_[i].reshape(50,50)
    plt.title("Eigenface " + str(i+1))
    plt.imshow(eigenfaces, cmap='bone')
plt.show()


## IV: Recreate images

In [ ]:
#for every dimension recreate a random image for each of the 5 sets of images
#and plot the initial image and the reconstructed image
#dictionary to store the initial image, and the reconstructed image for both dimensions

dimensions = [9, 30]
initial_reconstructed = {}

for set_number in range(1, 6):
    X_set, y_set = loadImages("/home/eva/Documents/machine_learning/project2/faces", "Set_" + str(set_number))
    random_image = np.random.randint(0, X_set.shape[0])  # Choose a random image
     
    for i, d in enumerate(dimensions):
        pca = PCA(n_components=d, whiten=True)  # Apply PCA with the current dimension
        X_train_pca = pca.fit_transform(X_set)  # Fit PCA with the training data
        original_image = normalization(X_set[random_image])  # Normalize the original image
    
        transformed_image = pca.transform(original_image.reshape(1, -1))  # Transform the image using the fitted PCA model
        reconstructed_image = pca.inverse_transform(transformed_image).reshape(50, 50)  # Reconstruct the image using the fitted PCA model
        reconstructed_image += np.mean(X_set1)  # Denormalize the reconstructed image
        initial_reconstructed[str(set_number) + "_" + str(d) ] = [original_image, reconstructed_image]

        plt.subplot(len(dimensions), 2, i*2 + 1)
        plt.title("Initial image (Dimension " + str(d) + ")")
        plt.imshow(original_image.reshape(50, 50), cmap='gray')

        plt.subplot(len(dimensions), 2, i*2 + 2)
        plt.title("Reconstructed image (Dimension " + str(d) + ")")
        plt.imshow(reconstructed_image, cmap='gray')

    plt.tight_layout()
    plt.show()


H ανακατασκευή της εικόνας από το πρώτο σετ, το οποίο χρησιμοποιήθηκε για την εκπαίδευση είναι η πιο ακριβής, Στο 2ο σετ η ποιότητα είναι λίγο χαμηλότερη, αλλά αρκετά υψηλή. Με την αυξηση των διαστάσεων σε 30 απο 9 οι ανακατασκευασμένες εικόνες περιέχουν περισσότερες λεπτομέρειες.

Στο τρίτο σετ εικόνων, η ποιότητα της ανακατασκευής αρχίζει να χειροτερεύει, ιδιαίτερα για d=9 όπου είναι αρκετά χαμηλή. Ωστόσο, για d=30 παρατηρούμε βελτίωση και εμφανίζονται περισσότερα χαρακτηριστικά του προσώπου, τα οποία είναι παρόμοια με αυτά του αρχικού προσώπου στην αρχική εικόνα.


???????????????????????????????????
Τέλος, στο τέταρτο και πέμπτο σετ εικόνων, η ανακατασκευή για d=9 δεν είναι καλή και πλησιάζει το "average" mean πρόσωπο του training set, καθώς δεν υπάρχει αρκετή πληροφορία. Ωστόσο, για d=30 παρατηρείται κάποια βελτίωση και υπάρχει περισσότερη λεπτομέρεια, αλλά παραμένει μακριά από την αρχική εικόνα. Αυτό είναι αναμενόμενο λόγω των διαφορετικών συνθηκών φωτισμού που υπάρχουν σε αυτά τα δύο τελευταία υποσύνολα εικόνων, τα οποία είναι τελείως διαφορετικά από το σετ 1 που χρησιμοποιήθηκε για την εκπαίδευση της μεθόδου.

## V:9 singular vectors

In [ ]:
#plot the 9 singular vectors, for the first set of images after applying SVD
U, S, V = np.linalg.svd(X_set1, full_matrices=False)

#plot the top 9 eigenfaces
print("Eigenvectors")
plt.figure(figsize=(20, 20))
for i in range(1,10):
  plt.subplot(1, 9, i)
  plt.title('EV %d' %i, fontweight ="light")
  plt.imshow(pca.components_[i-1].reshape(50, 50), cmap='bone')
plt.show()

#plot the top 9 singular vectors
print("\nSingular vectors")
plt.figure(figsize=(20, 20))
for i in range(1,10):
  plt.subplot(1, 9, i)
  plt.title('SV %d' %(i-1), fontweight ="light")
  plt.imshow(V[i-1,:].reshape(50, 50), cmap='bone')
plt.show()

Στα δεδομένα του PCA έχει γίνει normalization, όπου έχει αφαιρεθεί η μέση τιμή και έχει διαιρεθεί με την τυπική απόκλιση. Άρα από τα eigenvectors λείπει αυτη η πληροφορία, και οι διαφορές ανάμεσα στις εικόνες που προκύπτουν με τις δύο μεθόδους, οφείλονται σε αυτό.

# Classification using SVMs

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
dataset = fetch_openml('mnist_784', version = '1', parser='auto')
#transform every image to a 28x28 = 784 matrix
print(dataset.data.shape)
# Separate the features (X) and labels (y)
X, y = dataset['data'], dataset['target']

# Normalize the features to the range [0, 1]
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

## Linear SVM vs RBF kernel

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=60000, test_size=10000, random_state=42)

# Define the parameter combinations to evaluate
kernel_types = ['linear', 'rbf']
C_values = [0.1, 1, 10]
gamma_values = [0.1, 1, 10]

# Initialize variables to store the best parameters and accuracy
best_accuracy = 0
best_kernel = ''
best_C = 0
best_gamma = 0

# Iterate over parameter combinations
for kernel in kernel_types:
    for C in C_values:
        for gamma in gamma_values:
            # Create an SVM classifier with the current parameters
            svm = SVC(kernel=kernel, C=C, gamma=gamma)
            
            # Train the classifier
            svm.fit(X_train, y_train)
            
            # Predict labels for the test set
            y_pred = svm.predict(X_test)
            
            # Calculate the accuracy
            accuracy = accuracy_score(y_test, y_pred)
            
            # Check if the current combination has better accuracy
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_kernel = kernel
                best_C = C
                best_gamma = gamma

# Print the best parameter values and accuracy
print("Best kernel:", best_kernel)
print("Best C value:", best_C)
print("Best gamma value:", best_gamma)
print("Best accuracy:", best_accuracy)